This script produces a table of PMC paper IDs matched to the SRA or GEO datasets they contain, and metadata about those datasets.

In [1]:
# import required modules
import numpy as np
import pandas as pd
from collections import Counter
import statistics

STEP I: Clean the data

In [2]:
# import GEO reference data

# import table with accession, platform, and series
geoAPS = pd.read_csv('../geo_sample.csv')

# import table with datasets IDs
geoDS = pd.read_csv('../geo_series_table.csv', low_memory = False)
geoDS.columns = ['Series', 'Title', 'Series Type', 'Taxonomy', 
                 'Sample Count', 'Datasets', 'Supplementary Types', 
                 'Supplementary Links', 'PubMed ID', 'SRA Accession', 
                 'Contact', 'Release Date']

# add datasets column by merging
allData = pd.merge(geoAPS, geoDS, how = 'outer', on = 'Series')
geoReference = allData[['Series', 'Accession', 'Platform', 'Datasets']].drop_duplicates()

geoReference

,Series,Accession,Platform,Datasets
0,GSE506,GSM1,GPL4,NaN
1,GSE506,GSM2,GPL4,NaN
2,GSE462,GSM3,GPL5,NaN
3,GSE462,GSM4,GPL5,NaN
4,GSE462,GSM5,GPL5,NaN
...,...,...,...,...
3280011,GSE136775,NaN,NaN,NaN
3280012,GSE136776,NaN,NaN,NaN
3280013,GSE137458,NaN,NaN,NaN
3280014,GSE137562,NaN,NaN,NaN


In [3]:
# import SRA reference data
sraReference = pd.read_csv('../sraIDfull.csv', error_bad_lines = False, low_memory=False, quoting=3)
sraReference = sraReference[['SRAStudy', 'Run', 'Experiment', 
                             'BioProject', 'Submission', 'Sample']].drop_duplicates()
sraReference

,SRAStudy,Run,Experiment,BioProject,Submission,Sample
0,ERP000767,ERR3550121,ERX3556726,PRJEB2600,ERA2143351,ERS3773255
1,ERP000767,ERR3550142,ERX3556747,PRJEB2600,ERA2143351,ERS3773255
2,ERP000767,ERR3550147,ERX3556752,PRJEB2600,ERA2143351,ERS3773263
3,ERP000767,ERR3550125,ERX3556730,PRJEB2600,ERA2143351,ERS3773263
4,ERP000767,ERR3550137,ERX3556742,PRJEB2600,ERA2143351,ERS3773252
...,...,...,...,...,...,...
7676058,SRP002493,SRR049674,SRX020916,NaN,SRA017695,SRS073299
7676059,SRP002493,SRR049675,SRX021028,NaN,SRA017695,SRS073299
7676060,SRP002493,SRR049677,SRX021029,NaN,SRA017695,SRS073299
7676061,SRP002064,SRR042496,SRX020084,PRJNA46359,DIEGO,SRS011854


In [4]:
# import data scraped from PubMed
# this data was scraped from XML files on the hoffman2 cluster: /u/scratch/n/nikodm/pmcOA/

pmcData = pd.read_csv('../data_tables/preFilterMatrix.csv').drop_duplicates()
pmcData

,journal,pmc_ID,accession
0,Alzheimers_Res_Ther,PMC3707052,GSM1
1,Alzheimers_Res_Ther,PMC3706879,GSE45534
5,Alzheimers_Res_Ther,PMC4255636,GSE5281
6,Alzheimers_Res_Ther,PMC4731966,GSE67036
7,Alzheimers_Res_Ther,PMC4731966,GSE1297
...,...,...,...
144400,Dis_Markers,PMC6589301,GSE62254
144401,Dis_Markers,PMC3834650,GDS1059
144403,Dis_Markers,PMC3834650,GDS1
144404,Case_Rep_Hematol,PMC6462343,PRJNA437812


In [5]:
# Merge GEO accessions with reference data, convert to Series

# match series first
se = pd.merge(pmcData, geoReference['Series'].drop_duplicates(), how = 'left',
            left_on = 'accession', right_on = 'Series')
pmcData = se.rename(columns = {'Series': 'Series_result'})

# match each other style of GEO ID
for col in ['Accession', 'Platform', 'Datasets']:
    pmcData = pd.merge(pmcData, geoReference[['Series', col]].drop_duplicates(subset = col), how = 'left', 
            left_on = 'accession', right_on = col)
    label = col + '_result'
    pmcData = pmcData.rename(columns = {'Series': label})

# combine all GEO series match columns into one aggregate GEO series column, clean up
pmcData['geoSeries'] = pmcData['Series_result'].fillna(pmcData['Accession_result']).fillna(pmcData['Platform_result']).fillna(pmcData['Datasets_result'])
pmcData = pmcData.drop(labels = ['Accession', 'Platform', 'Datasets',
                           'Series_result', 'Accession_result',
                           'Platform_result', 'Datasets_result'], axis = 1)
pmcData_geoMerged = pmcData
pmcData_geoMerged

,journal,pmc_ID,accession,geoSeries
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534
2,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281
3,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036
4,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297
...,...,...,...,...
78615,Dis_Markers,PMC6589301,GSE62254,GSE62254
78616,Dis_Markers,PMC3834650,GDS1059,GSE2191
78617,Dis_Markers,PMC3834650,GDS1,NaN
78618,Case_Rep_Hematol,PMC6462343,PRJNA437812,NaN


In [6]:
# merge SRA accessions with reference data, convert to Study

# match SRA Study IDs first
st = pd.merge(pmcData_geoMerged, sraReference['SRAStudy'].drop_duplicates(), how = 'left',
            left_on = 'accession', right_on = 'SRAStudy')
pmcData = st.rename(columns = {'SRAStudy': 'Study_result'})

# match every other style of SRA ID
for col in ['Run', 'Experiment', 'BioProject', 'Submission', 'Sample']:
    pmcData = pd.merge(pmcData, sraReference[['SRAStudy', col]].drop_duplicates(subset = [col]), how = 'left',
                      left_on = 'accession', right_on = col)
    label = col + '_result'
    pmcData = pmcData.rename(columns = {'SRAStudy': label})

# combine all SRA Study matches into one aggregate column, clean up
pmcData['sraStudy'] = pmcData['Study_result'].fillna(pmcData['Run_result']).fillna(pmcData['Experiment_result']).fillna(pmcData['BioProject_result']).fillna(pmcData['Submission_result']).fillna(pmcData['Sample_result'])
pmcData = pmcData.drop(labels = ['Run', 'Experiment', 'BioProject', 'Submission', 'Sample',
                                'Study_result', 'Run_result', 'Experiment_result', 'BioProject_result',
                                'Submission_result', 'Sample_result'], axis = 1)

pmcData_sraMerged = pmcData
pmcData_sraMerged

,journal,pmc_ID,accession,geoSeries,sraStudy
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506,NaN
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534,NaN
2,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281,NaN
3,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036,NaN
4,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297,NaN
...,...,...,...,...,...
78615,Dis_Markers,PMC6589301,GSE62254,GSE62254,NaN
78616,Dis_Markers,PMC3834650,GDS1059,GSE2191,NaN
78617,Dis_Markers,PMC3834650,GDS1,NaN,NaN
78618,Case_Rep_Hematol,PMC6462343,PRJNA437812,NaN,SRP135146


In [7]:
# combine GEO Series hits and SRA study hits into one converted accession column
pmcData['converted_accession'] = pmcData_sraMerged['geoSeries'].fillna(pmcData_sraMerged['sraStudy'])
pmcData = pmcData.drop(labels = ['geoSeries', 'sraStudy'], axis = 1)
pmcData

,journal,pmc_ID,accession,converted_accession
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534
2,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281
3,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036
4,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297
...,...,...,...,...
78615,Dis_Markers,PMC6589301,GSE62254,GSE62254
78616,Dis_Markers,PMC3834650,GDS1059,GSE2191
78617,Dis_Markers,PMC3834650,GDS1,NaN
78618,Case_Rep_Hematol,PMC6462343,PRJNA437812,SRP135146


In [8]:
# clean out erroneous converted_accession entries, and rows that didn't map to a converted_accession
w = []
for a in pmcData['converted_accession']:
    if(type(a) == str):
        if a[0:3] != 'GSE' and a[0:3] != 'GPL' and a[1:3] != 'RP' and a not in w:
                w.append(a)
w.append(np.NaN)
        
pmcData = pmcData[~pmcData.converted_accession.isin(w)]
pmcData

,journal,pmc_ID,accession,converted_accession
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534
2,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281
3,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036
4,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297
...,...,...,...,...
78614,Dis_Markers,PMC6589307,GSE14795,GSE14795
78615,Dis_Markers,PMC6589301,GSE62254,GSE62254
78616,Dis_Markers,PMC3834650,GDS1059,GSE2191
78618,Case_Rep_Hematol,PMC6462343,PRJNA437812,SRP135146


Step II: Collect all desired factors about each SRA or GEO dataset, and add them to the table

In [9]:
# Define functions to convert dates to a universal format
def strToMonth(m):
    if(m == 'Jan'):
        return '01'
    elif(m == 'Feb'):
        return '02'
    elif(m == 'Mar'):
        return '03'
    elif(m == 'Apr'):
        return '04'
    elif(m == 'May'):
        return '05'
    elif(m == 'Jun'):
        return '06'
    elif(m == 'Jul'):
        return '07'
    elif(m == 'Aug'):
        return '08'
    elif(m == 'Sep'):
        return '09'
    elif(m == 'Oct'):
        return '10'
    elif(m == 'Nov'):
        return '11'
    elif(m == 'Dec'):
        return '12'
    else:
        return(np.NaN)
    
def convGEODate(d):
    if(type(d) == str):
        mon = strToMonth(d[0:3])
        day = d[4:6]
        yr = d[8:12]
        return yr + '-' + mon + '-' + day
    else:
        return np.NaN

# def convSRADate(d):
#     return d[0:10]

In [10]:
# import SRA attribute data
# CAUTION: large file! Time delay on import...

sraAttributes = pd.read_csv('../sra_complete_runs.csv', error_bad_lines = False, low_memory=False)    

In [11]:
# Convert dates to a universal format by ignoring time of day
pd.set_option('display.max_columns', 50)
sraAttributes['ReleaseDate'] = sraAttributes['ReleaseDate'].str[0:10]
sraAttributes

,Run,ReleaseDate,LoadDate,spots,bases,spots_with_mates,avgLength,size_MB,AssemblyName,download_path,Experiment,LibraryName,LibraryStrategy,LibrarySelection,LibrarySource,LibraryLayout,InsertSize,InsertDev,Platform,Model,SRAStudy,BioProject,Study_Pubmed_id,ProjectID,Sample,BioSample,SampleType,TaxID,ScientificName,SampleName,g1k_pop_code,source,g1k_analysis_group,Subject_ID,Sex,Disease,Tumor,Affection_Status,Analyte_Type,Histological_Type,Body_Site,CenterName,Submission,dbgap_study_accession,Consent,RunHash,ReadHash
0,ERR3550121,2019-09-25,NaN,0,0,0,0,0,NaN,NaN,ERX3556726,NaN,WGS,RANDOM,GENOMIC,PAIRED,250,0.0,ILLUMINA,Illumina HiSeq 2000,ERP000767,PRJEB2600,NaN,204335,ERS3773255,NaN,simple,0,Leishmania infantum,0c4ac31a-dec5-11e9-ba7b-68b599768938,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,UOC-CCG,ERA2143351,NaN,public,NaN,NaN
1,ERR3550142,2019-09-25,NaN,0,0,0,0,0,NaN,NaN,ERX3556747,NaN,WGS,RANDOM,GENOMIC,PAIRED,250,0.0,ILLUMINA,Illumina HiSeq 2000,ERP000767,PRJEB2600,NaN,204335,ERS3773255,NaN,simple,0,Leishmania infantum,0c4ac31a-dec5-11e9-ba7b-68b599768938,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,UOC-CCG,ERA2143351,NaN,public,NaN,NaN
2,ERR3550147,2019-09-25,NaN,0,0,0,0,0,NaN,NaN,ERX3556752,NaN,WGS,RANDOM,GENOMIC,PAIRED,250,0.0,ILLUMINA,Illumina HiSeq 2000,ERP000767,PRJEB2600,NaN,204335,ERS3773263,NaN,simple,0,Leishmania donovani,0e2d3a2c-dec6-11e9-a5d1-68b59976a384,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,UOC-CCG,ERA2143351,NaN,public,NaN,NaN
3,ERR3550125,2019-09-25,NaN,0,0,0,0,0,NaN,NaN,ERX3556730,NaN,WGS,RANDOM,GENOMIC,PAIRED,250,0.0,ILLUMINA,Illumina HiSeq 2000,ERP000767,PRJEB2600,NaN,204335,ERS3773263,NaN,simple,0,Leishmania donovani,0e2d3a2c-dec6-11e9-a5d1-68b59976a384,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,UOC-CCG,ERA2143351,NaN,public,NaN,NaN
4,ERR3550137,2019-09-25,NaN,0,0,0,0,0,NaN,NaN,ERX3556742,NaN,WGS,RANDOM,GENOMIC,PAIRED,250,0.0,ILLUMINA,Illumina HiSeq 2000,ERP000767,PRJEB2600,NaN,204335,ERS3773252,NaN,simple,0,Leishmania donovani,188c9a6e-deba-11e9-ba7b-68b599768938,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,UOC-CCG,ERA2143351,NaN,public,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7676058,SRR049674,2010-05-27,2014-05-27 02:57:13,35888370,1794418500,0,50,1504,NaN,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,SRX020916,VVWTA 0h A,FL-cDNA,RT-PCR,VIRAL RNA,SINGLE,0,0.0,ABI_SOLID,AB SOLiD System 3.0,SRP002493,NaN,2.0,0,SRS073299,SAMN00013848,simple,10245,Vaccinia virus,VACV,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NIAID-RML-RTS,SRA017695,NaN,public,1C9EE0CEFAE2352A0321B24EAB64EE18,753ED95CDA0B112FD877F38960613605
7676059,SRR049675,2010-05-27,2014-05-27 02:58:17,40264881,2013244050,0,50,1696,NaN,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,SRX021028,VVWTA 0.5A,FL-cDNA,RT-PCR,VIRAL RNA,SINGLE,0,0.0,ABI_SOLID,AB SOLiD System 3.0,SRP002493,NaN,2.0,0,SRS073299,SAMN00013848,simple,10245,Vaccinia virus,VACV,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NIAID-RML-RTS,SRA017695,NaN,public,9F96F35BE762574A084985FD9C918C28,392587111A738BC44AE8878E649D052A
7676060,SRR049677,2010-05-27,2014-05-27 02:58:58,40852259,2042612950,0,50,1732,NaN,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,SRX021029,VVWTA 1h A,FL-cDNA,RT-PCR,VIRAL RNA,SINGLE,0,0.0,ABI_SOLID,AB SOLiD System 3.0,SRP002493,NaN,2.0,0,SRS073299,SAMN00013848,simple,10245,Vaccinia virus,VACV,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NIAID-RML-RTS,SRA017695,NaN,public,22DCD0A93DAA207187EF6C6F6A835A55,416EBCA2B1A6AB70E4F778F16C36FCE5
7676061,SRR042496,2010-05-11,2012-01-19 16:52:13,47760,16505980,0,345,42,NaN,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,SRX020084,VirRnaMgMosqCeI,OTHER,RANDOM PCR,VIRAL RNA,SINGLE,0,0.0,LS454,454 GS FLX Titanium,SRP002064,PRJNA46359,2.0,46359,SRS011854,SAMN00012272,simple,1284618,environmental samples,VirRnaMgMosqCeI,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,SAN DIE

In [12]:
# import GEO attribute data and add Series column
geoAttributes = pd.read_csv('../geo_platforms_table.csv')

In [13]:
geoAttributes.rename(columns={'Accession':'Platform'}, inplace=True)
geoAttributes = pd.merge(geoAttributes, geoReference, how = 'left', on = 'Platform')
dates = []
for i in geoAttributes['Release Date']:
    dates.append(convGEODate(i))
geoAttributes['Release Date'] = dates
geoAttributes

,Platform,Title,Technology,Taxonomy,Data Rows,Samples Count,Series Count,Contact,Release Date,Series,Accession,Datasets
0,GPL4,SAGE:10:NlaIII:Homo sapiens,SAGE NlaIII,Homo sapiens,265577,580,58,GEO,2000-09-28,GSE506,GSM1,NaN
1,GPL4,SAGE:10:NlaIII:Homo sapiens,SAGE NlaIII,Homo sapiens,265577,580,58,GEO,2000-09-28,GSE506,GSM2,NaN
2,GPL4,SAGE:10:NlaIII:Homo sapiens,SAGE NlaIII,Homo sapiens,265577,580,58,GEO,2000-09-28,GSE10,GSM571,NaN
3,GPL4,SAGE:10:NlaIII:Homo sapiens,SAGE NlaIII,Homo sapiens,265577,580,58,GEO,2000-09-28,GSE10,GSM572,NaN
4,GPL4,SAGE:10:NlaIII:Homo sapiens,SAGE NlaIII,Homo sapiens,265577,580,58,GEO,2000-09-28,GSE10,GSM573,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2961228,GPL27549,Illumina HiSeq 2000 (Escherichia coli str. K-1...,high-throughput sequencing,Escherichia coli str. K-12 substr. DH10B,0,0,0,GEO,2019-09-30,NaN,NaN,NaN
2961229,GPL27550,Illumina HiSeq 4000 (Solanum chacoense),high-throughput sequencing,Solanum chacoense,0,0,0,GEO,2019-09-30,NaN,NaN,NaN
2961230,GPL27552,"Illumina HiSeq 2500 (Caenorhabditis elegans ,V...",high-throughput sequencing,"Caenorhabditis elegans ,Vibrio parahaemolyticus",0,0,0,GEO,2019-09-30,NaN,NaN,NaN
2961231,GPL27554,Illumina HiSeq 2000 (Trichoderma atroviride),high-throughput sequencing,Trichoderma atroviride,0,0,0,GEO,2019-09-30,NaN,NaN,NaN


In [14]:
# Add a column tagging each accession as GEO or SRA

repoList = []

for i in pmcData['converted_accession']:
    if(type(i) == str):
        if('GSE' in i):
            repoList.append('GEO')
        elif('SRP' in i or 'ERP' in i or 'DRP' in i):
            repoList.append('SRA')
        else:
            repoList.append(np.NaN)
    else:
        repoList.append(np.NaN)
        
pmcData['repository'] = repoList
pmcData

/Users/niko/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,journal,pmc_ID,accession,converted_accession,repository
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506,GEO
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534,GEO
2,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281,GEO
3,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036,GEO
4,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297,GEO
...,...,...,...,...,...
78614,Dis_Markers,PMC6589307,GSE14795,GSE14795,GEO
78615,Dis_Markers,PMC6589301,GSE62254,GSE62254,GEO
78616,Dis_Markers,PMC3834650,GDS1059,GSE2191,GEO
78618,Case_Rep_Hematol,PMC6462343,PRJNA437812,SRP135146,SRA


In [15]:
# add column for paper publish date
# this data was scraped from XML files on the hoffman2 cluster: /u/scratch/n/nikodm/pmcOA/

pmc_dates = pd.read_csv('../data_lists/postFilterDates.csv').drop_duplicates()

pmcData = pd.merge(pmcData, pmc_dates, how = 'left', on = 'pmc_ID')
pmcData = pmcData.rename(columns = {'date': 'pmc_date'})

pmcData

,journal,pmc_ID,accession,converted_accession,repository,pmc_date
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506,GEO,2013-4-18
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534,GEO,2013-5-14
2,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281,GEO,2014-11-2
3,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036,GEO,2016-1-28
4,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297,GEO,2016-1-28
...,...,...,...,...,...,...
73485,Dis_Markers,PMC6589307,GSE14795,GSE14795,GEO,2019-6-4
73486,Dis_Markers,PMC6589301,GSE62254,GSE62254,GEO,2019-6-2
73487,Dis_Markers,PMC3834650,GDS1059,GSE2191,GEO,2013-11-5
73488,Case_Rep_Hematol,PMC6462343,PRJNA437812,SRP135146,SRA,2019-3-31


In [16]:
# Get every factor we're interested in from our tables of GEO and SRA metadata...

# take a slice of the GEO and SRA attribute tables with only the info we want
slicedGEOAtt = geoAttributes[['Series', 'Release Date', 'Technology', 'Taxonomy']]
slicedGEOAtt.columns = ['converted_accession', 'geoRelease', 'geoHardware', 'geoSpecies']
slicedGEOAtt = slicedGEOAtt.drop_duplicates(subset = ['converted_accession'])

slicedSRAAtt = sraAttributes[['SRAStudy', 'ReleaseDate', 'Model', 
                              'LibraryStrategy', 'ScientificName', 
                              'bases', 'avgLength', 'Consent']]
slicedSRAAtt.columns = ['converted_accession', 'sraRelease', 'sraHardware', 
                        'sraLibrary_strategy', 'sraSpecies', 
                        'sraBases', 'sraAvg_length', 'sraAccess']
slicedSRAAtt = slicedSRAAtt.drop_duplicates(subset = ['converted_accession'])

In [17]:
# special case for GEO: make an educated guess on library strategy based on hardware
# These guesses are based on manually checking GEO series IDs that corresponded to various types of hardware

gc = Counter(geoAttributes['Technology'])
ls_guesses = pd.DataFrame.from_dict(gc, orient='index').reset_index()
ls_guesses.columns = ['hardware', 'use_count']
ls_guesses = ls_guesses.drop(labels = ['use_count'], axis = 1)

ls = []

for i in ls_guesses['hardware']:
    if(i == 'high-throughput sequencing'):
        ls.append('RNA-Seq')
    elif(i == 'SAGE NlaIII' or i == 'spotted DNA/cDNA' or i == 'SAGE Sau3A' 
         or i == 'in situ oligonucleotide' or i == 'spotted oligonucleotide' 
         or i == 'antibody' or i == 'MPSS' or i == 'oligonucleotide beads' 
         or i == 'RT-PCR' or i == 'mixed spotted oligonucleotide/cDNA' 
         or i == 'spotted peptide or protein'):
        ls.append('Expression_Array')
    else:
        ls.append(np.NaN)
        
ls_guesses.loc[:,'geoLibrary_strategy'] = ls
ls_guesses

,hardware,geoLibrary_strategy
0,SAGE NlaIII,Expression_Array
1,spotted DNA/cDNA,Expression_Array
2,SAGE Sau3A,Expression_Array
3,in situ oligonucleotide,Expression_Array
4,spotted oligonucleotide,Expression_Array
5,antibody,Expression_Array
6,MS,NaN
7,SARST,NaN
8,SAGE RsaI,NaN
9,other,NaN


In [18]:
# merge SRA attributes onto pmcData table
mergedSRA = pd.merge(pmcData, slicedSRAAtt, how = 'left', on = 'converted_accession')
mergedSRA = mergedSRA.drop_duplicates()

# merge GEO attributes onto table of pmcData + SRA Attributes
allFactors = pd.merge(mergedSRA, slicedGEOAtt, how = 'left', on = 'converted_accession')
allFactors = pd.merge(allFactors, ls_guesses, how = 'left', left_on = 'geoHardware', right_on = 'hardware')

allFactors = allFactors.dropna(subset = ['converted_accession'])

In [19]:
# clean up columns with factor for both SRA and GEO, rearrange columns
allFactors['species'] = allFactors['sraSpecies'].fillna(allFactors['geoSpecies'])
allFactors = allFactors.drop(labels = ['sraSpecies', 'geoSpecies'], axis = 1)

allFactors['hardware'] = allFactors['sraHardware'].fillna(allFactors['geoHardware'])
allFactors = allFactors.drop(labels = ['sraHardware', 'geoHardware'], axis = 1)

allFactors['library_strategy'] = allFactors['sraLibrary_strategy'].fillna(allFactors['geoLibrary_strategy'])
allFactors = allFactors.drop(labels = ['sraLibrary_strategy', 'geoLibrary_strategy'], axis = 1)

allFactors['repository_date'] = allFactors['sraRelease'].fillna(allFactors['geoRelease'])
allFactors = allFactors.drop(labels = ['sraRelease', 'geoRelease'], axis = 1)

cols = ['journal', 'pmc_ID', 'accession', 'converted_accession', 'repository', 
        'pmc_date', 'repository_date', 'species', 
        'hardware', 'library_strategy', 'sraAvg_length', 'sraBases', 'sraAccess']

allFactors = allFactors[cols]
allFactors

,journal,pmc_ID,accession,converted_accession,repository,pmc_date,repository_date,species,hardware,library_strategy,sraAvg_length,sraBases,sraAccess
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506,GEO,2013-4-18,2000-09-28,Homo sapiens,SAGE NlaIII,Expression_Array,NaN,NaN,NaN
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534,GEO,2013-5-14,2004-05-25,Mus musculus,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN
2,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281,GEO,2014-11-2,2003-11-07,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN
3,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036,GEO,2016-1-28,2013-05-06,Rattus norvegicus,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN
4,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297,GEO,2016-1-28,2002-03-11,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73485,Dis_Markers,PMC6589307,GSE14795,GSE14795,GEO,2019-6-4,2002-03-11,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN
73486,Dis_Markers,PMC6589301,GSE62254,GSE62254,GEO,2019-6-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73487,Dis_Markers,PMC3834650,GDS1059,GSE2191,GEO,2013-11-5,2009-03-16,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN
73488,Case_Rep_Hematol,PMC6462343,PRJNA437812,SRP135146,SRA,2019-3-31,2018-03-13,human gut metagenome,Illumina MiSeq,AMPLICON,602.0,17743348.0,public


Step III: Conduct analysis on this table, producing new metrics

In [20]:
# add a column with the total use count of each converted_accession ID

# first, create a new dataframe with no double-counted accessions
# justAcc includes all unique pairings of PMC ID and Series/Study accession
justAcc = allFactors.loc[:, ['pmc_ID', 'converted_accession']]
justAcc = justAcc.drop_duplicates()

rc = Counter(justAcc['converted_accession'])
reuse_counts = pd.DataFrame.from_dict(rc, orient = 'index').reset_index()
reuse_counts.columns = ['converted_accession', 'total_use_count']
reuse_counts

,converted_accession,total_use_count
0,GSE506,64
1,GSE45534,1
2,GSE5281,43
3,GSE67036,1
4,GSE1297,32
...,...,...
35221,GSE85841,1
35222,GPL20115,1
35223,GSE73953,1
35224,GSE14795,1


In [21]:
# merge reuse counts onto table
allFactors = pd.merge(allFactors, reuse_counts, how = 'left', on = 'converted_accession')
allFactors

,journal,pmc_ID,accession,converted_accession,repository,pmc_date,repository_date,species,hardware,library_strategy,sraAvg_length,sraBases,sraAccess,total_use_count
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506,GEO,2013-4-18,2000-09-28,Homo sapiens,SAGE NlaIII,Expression_Array,NaN,NaN,NaN,64
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534,GEO,2013-5-14,2004-05-25,Mus musculus,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,1
2,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281,GEO,2014-11-2,2003-11-07,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,43
3,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036,GEO,2016-1-28,2013-05-06,Rattus norvegicus,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,1
4,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297,GEO,2016-1-28,2002-03-11,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73485,Dis_Markers,PMC6589307,GSE14795,GSE14795,GEO,2019-6-4,2002-03-11,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,1
73486,Dis_Markers,PMC6589301,GSE62254,GSE62254,GEO,2019-6-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
73487,Dis_Markers,PMC3834650,GDS1059,GSE2191,GEO,2013-11-5,2009-03-16,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,5
73488,Case_Rep_Hematol,PMC6462343,PRJNA437812,SRP135146,SRA,2019-3-31,2018-03-13,human gut metagenome,Illumina MiSeq,AMPLICON,602.0,17743348.0,public,1


In [22]:
# Tag each paper as either a generator or a reuser

# take each converted accession along with the first date it appears in a paper
justPap = allFactors.loc[:, ['converted_accession', 'pmc_date']].drop_duplicates()
justPap = justPap.groupby('converted_accession', as_index = False).agg({'pmc_date': 'min'})
justPap = justPap.rename(columns = {'pmc_date':'min_pmc_date'})

# label all of the usages of these datasets on these dates with a 'G' (generators)
justPap.loc[:,'reuse_role'] = 'G'
justPap

,converted_accession,min_pmc_date,reuse_role
0,DRP000010,2010-4-27,G
1,DRP000017,2011-10-9,G
2,DRP000018,2011-10-9,G
3,DRP000019,2011-10-9,G
4,DRP000020,2011-10-9,G
...,...,...,...
35221,SRP212249,2019-7-08,G
35222,SRP212328,2019-7-03,G
35223,SRP216761,2017-11-21,G
35224,SRP221037,2019-3-21,G


In [23]:
# merge these back onto the original table, matching dates back to papers
allFactors = pd.merge(allFactors, justPap, how = 'left', 
                      left_on = ['converted_accession', 'pmc_date'], 
                      right_on = ['converted_accession', 'min_pmc_date'])

# label every usage of a dataset that isn't 'G' (generator) as 'R' (reuser)
allFactors['allR'] = 'R'
allFactors['reuse_role'] = allFactors['reuse_role'].fillna(allFactors['allR'])

# drop redundant columns
allFactors = allFactors.drop(labels = ['min_pmc_date', 'allR'], axis = 1)

allFactors

,journal,pmc_ID,accession,converted_accession,repository,pmc_date,repository_date,species,hardware,library_strategy,sraAvg_length,sraBases,sraAccess,total_use_count,reuse_role
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506,GEO,2013-4-18,2000-09-28,Homo sapiens,SAGE NlaIII,Expression_Array,NaN,NaN,NaN,64,R
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534,GEO,2013-5-14,2004-05-25,Mus musculus,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,1,G
2,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281,GEO,2014-11-2,2003-11-07,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,43,R
3,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036,GEO,2016-1-28,2013-05-06,Rattus norvegicus,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,1,G
4,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297,GEO,2016-1-28,2002-03-11,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,32,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73485,Dis_Markers,PMC6589307,GSE14795,GSE14795,GEO,2019-6-4,2002-03-11,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,1,G
73486,Dis_Markers,PMC6589301,GSE62254,GSE62254,GEO,2019-6-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,R
73487,Dis_Markers,PMC3834650,GDS1059,GSE2191,GEO,2013-11-5,2009-03-16,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,5,G
73488,Case_Rep_Hematol,PMC6462343,PRJNA437812,SRP135146,SRA,2019-3-31,2018-03-13,human gut metagenome,Illumina MiSeq,AMPLICON,602.0,17743348.0,public,1,G


In [24]:
# save to .csv
allFactors.to_csv('../data_tables/metadataMatrix.csv', index = False)

In [25]:
# # define function returning corresponding metadata or Study/Series ID given any information on a dataset
# def grabRelated(table, key, key_col, out_col):
#     indices = table.index[table[key_col] == key].tolist()
#     if(len(indices) > 0):
#         allHits = table.loc[indices][[out_col]]
#         modeHits = allHits.mode()
#         if(modeHits.empty):
#             return np.NaN
#         else:
#             return modeHits.iloc[0,0]
#     else:
#         return np.NaN

# # define functions to perform QC on accessions, checking if they exist in the references
# def sraChecksOut(acc, col):
#     if(acc in sraReference[col].tolist()):
#         return True
#     else:
#         return False
    
# def geoChecksOut(acc, col):
#     if(acc in geoReference[col].tolist()):
#         return True
#     else:
#         return False

In [26]:
# # check every accession against the corresponding reference, and convert to STUDY/SERIES
# # add converted accessions to a new list, to be added as a column later
# # CAUTION: long input list, loop takes several hours to run

# converted_acc = []

# for acc in pmcData['accession']:
#     style = ''
#     # assign a style to the accession, corresponds to column names
#     # SRA styles
#     if('SRP' in acc or 'ERP' in acc or 'DRP' in acc):
#         style = 'SRAStudy'
#     elif('SRR' in acc or 'ERR' in acc or 'DRR' in acc):
#         style = 'Run'
#     elif('SRX' in acc or 'ERX' in acc or 'DRX' in acc):
#         style = 'Experiment'
#     elif('PRJNA' in acc or 'PRJD' in acc or 'PRJEB' in acc):
#         style = 'BioProject'
#     elif('SRA' in acc or 'ERA' in acc or 'DRA' in acc):
#         style = 'Submission'
#     elif('SRS' in acc or 'ERS' in acc or 'DRS' in acc):
#         style = 'Sample'
#     # GEO styles
#     elif('GSE' in acc):
#         style = 'Series'
#     elif('GSM' in acc):
#         style = 'Accession'
#     elif('GPL' in acc):
#         style = 'Platform'
#     elif('GDS' in acc):
#         style = 'Datasets'
#     else:
#         style = ''
        
#     # for SRA accessions, check if they exist in the SRA reference
#     # ...and add corresponding STUDY ID to new column
#     if(style == 'SRAStudy' or style == 'Run' or style == 'Experiment' or 
#        style == 'BioProject' or style == 'Submission' or style == 'Sample'):
#         if(not sraChecksOut(acc, style)):
#             converted_acc.append(np.NaN)
#         else:
#             converted_acc.append(grabRelated(sraReference, acc, style, 'SRAStudy'))
#     # for GEO accessions, check if they exist in the GEO reference
#     # ...and add corresponding SERIES ID to new column
#     elif(style == 'Series' or style == 'Accession' or style == 'Platform' or style == 'Datasets'):
#         if(not geoChecksOut(acc, style)):
#             converted_acc.append(np.NaN)
#         else:
#             converted_acc.append(grabRelated(geoReference, acc, style, 'Series'))
#     # something doesn't belong, record as missing
#     else:
#         converted_acc.append(np.NaN)

# # add the converted accession list as a new column in PubMed data table
# # pd.set_option('display.min_rows', 50)
# pmcData.loc[:,'converted_accession'] = converted_acc
# pmcData = pmcData.drop_duplicates()
# pmcData

In [27]:
# save 'checkpoint' to CSV file
# pmcData.to_csv('postFilterMatrix.csv')

In [28]:
# # temporary cell saving time from big loop...

# pmcData = pd.read_csv('../data_tables/postFilterMatrix.csv')
# pmcData